In [1]:
import torch
import torch.nn as nn
import numpy as np

from src.data.simulation import Simulation

from src.costs.torch_cost_functions import (
    TorchB1HomogeneityCost,
    TorchB1HomogeneityCostSAR,
)
from src.costs.b1_homogenity_with_SAR import B1HomogeneityCostSAR
from src.optimizers.torch_optimizer import TorchOptimizer
from src.optimizers.optim_2 import HillClimbingOptimizer

In [2]:
sim = Simulation(
    "src/data/simulations/children_0_tubes_5_id_12346.h5", "src/data/antenna/antenna.h5"
)

Torch

In [3]:
# loss_1 = TorchB1HomogeneityCost()
loss_2 = TorchB1HomogeneityCostSAR()

In [4]:
optimizer = TorchOptimizer(cost_function=loss_2)
coil_config = optimizer.optimize(simulation=sim)
print(coil_config)

  0%|          | 3/1000 [00:00<01:40,  9.91it/s]

Iteration 0: Loss = -0.7687695026397705


 10%|█         | 102/1000 [00:09<01:21, 11.06it/s]

Iteration 100: Loss = -1.7376679182052612


 20%|██        | 202/1000 [00:18<01:11, 11.21it/s]

...

Iteration 800: Loss = -1.7505388259887695


 90%|█████████ | 902/1000 [01:22<00:09, 10.86it/s]

Iteration 900: Loss = -1.749538540840149


100%|██████████| 1000/1000 [01:31<00:00, 10.93it/s]

CoilConfig(phase=array([-0.28317672, -1.5708698 , -0.0804247 ,  0.80682796,  1.2900586 ,
       -1.2419237 , -1.0786561 , -0.657999  ], dtype=float32), amplitude=array([ 0.2662608 , -0.04654339, -0.137831  , -0.35575658, -0.44317976,
        0.48881164,  0.5395383 ,  0.354603  ], dtype=float32))


In [ ]:
loss_hill = B1HomogeneityCostSAR()
hill_optimizer = HillClimbingOptimizer(cost_function=loss_hill)
hill_optimizer.optimize(simulation=sim)